In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Read the data
data_path = Path.cwd().parent / "assets" / "datasets" / "processed" / "ree-demand-profiles.feather"
df = pd.read_feather(data_path)

In [3]:
df.head()

,p20td,p30td,p30tdve,ref_demand_2023_mw,timestamp
0,0.000136,0.000086,0.000084,25418,2023-01-01 01:00:00
1,0.000116,0.000081,0.000066,23582,2023-01-01 02:00:00
2,0.000100,0.000078,0.000050,22153,2023-01-01 03:00:00
3,0.000091,0.000077,0.000047,21272,2023-01-01 04:00:00
4,0.000085,0.000077,0.000058,20844,2023-01-01 05:00:00


In [4]:
from prophet import Prophet
from prophet.plot import plot_plotly
import datetime as dt

# prophet expects a dataframe with a column named "ds" and a column named "y"
df = df.rename(columns={"timestamp": "ds", "ref_demand_2023_mw": "y"})

m = Prophet()
m.fit(df)

16:02:58 - cmdstanpy - INFO - Chain [1] start processing
16:03:03 - cmdstanpy - INFO - Chain [1] done processing


In [5]:
# create a future dataframe
n_future_days = 90

ds = df["ds"].max()
future_dates = []

for _ in range(n_future_days):
	ds = ds + dt.timedelta(days=1)
	future_dates.append(ds)

df_future = pd.DataFrame({"ds": future_dates})

In [6]:
# predict next 90 days
forecast = m.predict(df_future)

In [8]:
# show the forecast
fig = plot_plotly(m, forecast, xlabel="Date", ylabel="Demand (MW)")
fig

In [19]:
# save results
out_dir = Path.cwd().parent / "out" 
fig.write_image(out_dir / "forecast_ree_90d.png")